In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import csv

In [3]:
# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By

In [4]:

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [5]:
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

Fase 2: Extracción de Detalles de Películas con Selenium
Una vez que tengan una lista de películas de la API, el siguiente paso es obtener información más detallada sobre ellas. Utilizaras la herramienta de automatización web Selenium para navegar a sitios web de reseñas de películas (IMDB y Rotten Tomatoes) y extraer detalles como calificaciones, actores y directores. En concreto deberás extraer información sobre:

Puntuacion de IMDB (en caso de que la tenga).

Puntuación de Rotten Tomatoes (Tomatometer).

Dirección (director/a o directore/as de cada película).

Gionistas (de cada película).

Argumento.

Duración (en minutos).

Nombre de la película

Probar un try para el buscador y en except break, asi me devuelve los resultados en csv hasta el momento?

In [114]:



def llamar_info_peli(lista_id,driver,archivo_csv):
  # Creo lista de tuplas tal y como pide el enunciado para guardar los datos extraidos 

  lista_tuplas_pelis=[]


  # id de peliculas por las que vamos a iterar para extraer la info


  for id in lista_id:
      
      # Creo la lista donde voy a ir guardando la info de cada peli, que despues convertiremos en tupla 

      lista_datos_imdb=[]
      
      lista_datos_imdb.append(id)

      try: 

        sleep(3)

        busqueda = driver.find_element("css selector","#suggestion-search")

        busqueda.clear()

        busqueda.send_keys(id, Keys.ENTER)

        # Extraemos la informacion que necesitamos

        try:

          nombre_pelicula=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/h1/span').text
          #nombre_pelicula=driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > h1 > span").text 
          lista_datos_imdb.append(nombre_pelicula)

        except:
          go_homepage= driver.find_element("xpath",'//*[@id="__next"]/main/div[2]/div/div[2]/div/div[1]/a').click()
          lista_datos_imdb.append("No encontrada")
          print("No se ha encontrado esta película en IMDB")


        # DIRECCION: extraemos la informacion de los directores, que pueden ser 1 o varios
        # Creamos una lista donde iremos agregando los diferentes directores en base a las etiquetas correspondientes

        #direccion_peli=[]

        try: 
          
          director1= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/ul/li[1]/div/ul/li[1]/a').text
          lista_datos_imdb.append(director1) 

        except:  

            try:
                   
              director1= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/ul/li[1]/div/ul/li/a').text
              lista_datos_imdb.append(director1)

            except:

              try:
               
               director2= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[1]/div').text
               lista_datos_imdb.append(director2)     
              except:
                lista_datos_imdb.append("No encontrado")
                                            
        # Guionistas

        try:
          
          guionistas_peli= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/ul/li[2]/div').text
          guionistas_bien=re.sub(r'([a-z])([A-Z])', r'\1, \2', guionistas_peli)
          lista_datos_imdb.append(guionistas_bien)

        except:
            try:
              guionistas_peli= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/ul/li[2]/div').text
              guionistas_bien=re.sub(r'([a-z])([A-Z])', r'\1, \2', guionistas_peli)
              lista_datos_imdb.append(guionistas_bien)

            except:
                try:
                  guionistas_peli=driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[2]/div').text
                  guionistas_bien=re.sub(r'([a-z])([A-Z])', r'\1, \2', guionistas_peli)
                  lista_datos_imdb.append(guionistas_bien) 
                except:
                  lista_datos_imdb.append("No encontrado")



        # Argumento
        # Para argumento hay varias etiquetas, una para cada tamaño de pantalla o dispositivo, por lo que hay que escoger la de pantalla grande
        try:
          argumento_peli=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]').text
          lista_datos_imdb.append(argumento_peli) 
        except:
           lista_datos_imdb.append("No encontrado")
          

      
        # Duracion pelicula:

        try: 
          duracion_peli=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[3]').text
          #duracion_peli= driver.find_element("css selector","__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(28) > div.sc-f65f65be-0.bBlII > ul > li:nth-child(1) > div").text

          # Encontrar los números en el formato de duración
          duracion_minutos= re.findall(r'\d+', duracion_peli)
              
          # Inicializar horas y minutos
          horas = 0
          minutos = 0

          # Asignar horas y minutos según los valores encontrados
          if 'h' in duracion_peli and 'min' in duracion_peli:
              horas = int(duracion_minutos[0])
              minutos = int(duracion_minutos[1])
          elif 'h' in duracion_peli:
              horas = int(duracion_minutos[0])
          elif 'min' in duracion_peli:
              minutos = int(duracion_minutos[0])
              
          # Calcular la duración total en minutos
          duracion_total_minutos = (horas * 60) + minutos
             
          
          lista_datos_imdb.append(duracion_total_minutos)

        except:

          try:  
            duracion_peli=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[2]').text
            
            # Encontrar los números en el formato de duración
            duracion_minutos= re.findall(r'\d+', duracion_peli)
                
            # Inicializar horas y minutos
            horas = 0
            minutos = 0

            # Asignar horas y minutos según los valores encontrados
            if 'h' in duracion_peli and 'min' in duracion_peli:
                  horas = int(duracion_minutos[0])
                  minutos = int(duracion_minutos[1])
            elif 'h' in duracion_peli:
                  horas = int(duracion_minutos[0])
            elif 'min' in duracion_peli:
                  minutos = int(duracion_minutos[0])
                
              # Calcular la duración total en minutos
            duracion_total_minutos = (horas * 60) + minutos

            lista_datos_imdb.append(duracion_total_minutos)
            
          except:
  
             lista_datos_imdb.append("No encontrado")
          
    
        # Puntuacion
        try:
          
          puntuacion_imdb= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]').text
          lista_datos_imdb.append(puntuacion_imdb.replace(",", "."))

        except:
        
          lista_datos_imdb.append("No encontrado")

      except Exception as null:
          print(f"Error al procesar la búsqueda para el ID {id}: {str(null)}")

    
    # Guardo todos los datos extraidos de cada peli en una tupla, que se incluye, a su vez, en la lista de tuplas de todas las peliculas

      lista_tuplas_pelis.append(tuple(lista_datos_imdb))

   # Guardo informacion en un csv

  with open(archivo_csv, 'w', newline='', encoding='utf-8') as archivo:
         
     escritor =csv.writer(archivo) 
     escritor.writerow(["Pelicula_id", "Nombre", "Director", "Guionista", "Argumento", "Duración", "Puntuación IMDb"])
     escritor.writerows(lista_tuplas_pelis)
  
   # Sale a pagina principal y vuelve a empezar bucle

  sleep(3)

  driver.back() 

  sleep(3)

  return lista_tuplas_pelis


Extraemos de los CSV de la API las listas por id

In [57]:
# Importamos el csv de Accion para extraer los ID y llamar a la funcion

datos_accion_csv=pd.read_csv(r"C:\Users\silvi\Desktop\Adalab\project-da-promo-H-module-2-team-Interstellar\API\CSVs\Películas_['Action']_2010-2024.csv") 
datos_comedia_csv=pd.read_csv(r"C:\Users\silvi\Desktop\Adalab\project-da-promo-H-module-2-team-Interstellar\API\CSVs\Películas_['Comedy']_2010-2025.csv")
datos_drama_csv=pd.read_csv(r"C:\Users\silvi\Desktop\Adalab\project-da-promo-H-module-2-team-Interstellar\API\CSVs\Películas_['Drama']_2010-2025.csv")

In [55]:
type(datos_accion_csv)

pandas.core.frame.DataFrame

In [58]:
# Quiero ver como se llaman las columnas del csv para buscar la columna de ID

print(datos_comedia_csv.columns)

Index(['Tipo', 'Género', 'Título', 'Mes', 'Año', 'ID_IMDB'], dtype='object')


In [59]:
# Extraemos los id y posteriormente los convertiremos a una lista

id_accion=datos_accion_csv["ID_IMDB"]
id_comedia=datos_comedia_csv["ID_IMDB"]
id_drama=datos_drama_csv["ID_IMDB"]

In [60]:
lista_id_accion=id_accion.tolist()
lista_id_comedia=id_comedia.tolist()
lista_id_drama=id_drama.tolist()

In [61]:
#  todos los id en una lista 

union_listas =[]
union_listas.extend(lista_id_accion)
union_listas.extend(lista_id_comedia)
union_listas.extend(lista_id_drama)

In [62]:
# cuento numero de elementos en la lista
print(len(union_listas))

15688


In [63]:
# elimino id duplicados duplicados

lista_id_set = set(union_listas)
print(len(lista_id_set))

14544


In [64]:
#lo convierto en lista

lista_id = list(lista_id_set)
len(lista_id)

14544

In [66]:
# Cada vez que lo ejecuto el orden de la lista es diferente!!

lista_id

['tt1800301',
 'tt1847719',
 'tt10270648',
 'tt14991492',
 'tt2363363',
 'tt25402886',
 'tt26686370',
 'tt26749149',
 'tt28600432',
 'tt11006570',
 'tt15248456',
 'tt10039168',
 'tt2637906',
 'tt15863566',
 'tt2048865',
 'tt10048402',
 'tt10021686',
 'tt12833220',
 'tt1646971',
 'tt3036654',
 'tt14998762',
 'tt10704730',
 'tt3236976',
 'tt13061874',
 'tt12385404',
 'tt12605454',
 'tt10493472',
 'tt1232828',
 'tt10365862',
 'tt11924222',
 'tt27042672',
 'tt1650461',
 'tt10069486',
 'tt19383308',
 'tt1905034',
 'tt13073642',
 'tt14060232',
 'tt13099072',
 'tt3325766',
 'tt11928632',
 'tt10716904',
 'tt13667552',
 'tt10218912',
 'tt2381249',
 'tt10044678',
 'tt14247286',
 'tt13067244',
 'tt15021154',
 'tt1590796',
 'tt10681656',
 'tt1649783',
 'tt10022482',
 'tt1801128',
 'tt3361856',
 'tt1155076',
 'tt1548001',
 'tt26007064',
 'tt1647458',
 'tt10151102',
 'tt3109188',
 'tt27708536',
 'tt10411852',
 'tt10284978',
 'tt14412366',
 'tt10447544',
 'tt11257188',
 'tt3687952',
 'tt1277403',
 't

Vamos a crear un CSV con la lista_id ejecutada en este momento, para no tener que ejecutar cada vez y de esta manera tener una lista FIJA y que no vaya cambiando en cada ejecución

In [67]:
df = pd.DataFrame(lista_id, columns=['ID_Pelicula'])

In [68]:
df

,ID_Pelicula
0,tt1800301
1,tt1847719
2,tt10270648
3,tt14991492
4,tt2363363
...,...
14539,tt11210110
14540,tt2397597
14541,tt1851085
14542,tt14856628


In [76]:

lista_id_definitiva_csv= "lista_id_definitiva.csv"

df.to_csv(lista_id_definitiva_csv, index=False)

Creamos sublistas para ir ejecutando poco a poco

In [72]:
sublista_1=lista_id[:1000] 
len(sublista_1) 


1000

Convertimos sublista en CSV

In [75]:
df_sublista=pd.DataFrame(sublista_1, columns=['ID_Pelicula'])
df_sublista

,ID_Pelicula
0,tt1800301
1,tt1847719
2,tt10270648
3,tt14991492
4,tt2363363
...,...
995,tt10106152
996,tt10363182
997,tt3857098
998,tt10095498


In [77]:
sublista_selenium='sublista_selenium.csv'
df_sublista.to_csv(sublista_selenium, index=False)

Traemos desde el CSV la sublista para ejecutarla en la funcion

In [106]:
df_sublista_1=pd.read_csv(r"C:\Users\silvi\Desktop\Adalab\project-da-promo-H-module-2-team-Interstellar\SELENIUM\sublista_selenium.csv") 

df_sublista_1

,ID_Pelicula
0,tt1800301
1,tt1847719
2,tt10270648
3,tt14991492
4,tt2363363
...,...
995,tt10106152
996,tt10363182
997,tt3857098
998,tt10095498


In [110]:
id_pelis=df_sublista_1["ID_Pelicula"]

id_pelis

0       tt1800301
1       tt1847719
2      tt10270648
3      tt14991492
4       tt2363363
          ...    
995    tt10106152
996    tt10363182
997     tt3857098
998    tt10095498
999     tt1067107
Name: ID_Pelicula, Length: 1000, dtype: object

In [112]:
lista_id_sublista=id_pelis.tolist() 

lista_id_sublista

['tt1800301',
 'tt1847719',
 'tt10270648',
 'tt14991492',
 'tt2363363',
 'tt25402886',
 'tt26686370',
 'tt26749149',
 'tt28600432',
 'tt11006570',
 'tt15248456',
 'tt10039168',
 'tt2637906',
 'tt15863566',
 'tt2048865',
 'tt10048402',
 'tt10021686',
 'tt12833220',
 'tt1646971',
 'tt3036654',
 'tt14998762',
 'tt10704730',
 'tt3236976',
 'tt13061874',
 'tt12385404',
 'tt12605454',
 'tt10493472',
 'tt1232828',
 'tt10365862',
 'tt11924222',
 'tt27042672',
 'tt1650461',
 'tt10069486',
 'tt19383308',
 'tt1905034',
 'tt13073642',
 'tt14060232',
 'tt13099072',
 'tt3325766',
 'tt11928632',
 'tt10716904',
 'tt13667552',
 'tt10218912',
 'tt2381249',
 'tt10044678',
 'tt14247286',
 'tt13067244',
 'tt15021154',
 'tt1590796',
 'tt10681656',
 'tt1649783',
 'tt10022482',
 'tt1801128',
 'tt3361856',
 'tt1155076',
 'tt1548001',
 'tt26007064',
 'tt1647458',
 'tt10151102',
 'tt3109188',
 'tt27708536',
 'tt10411852',
 'tt10284978',
 'tt14412366',
 'tt10447544',
 'tt11257188',
 'tt3687952',
 'tt1277403',
 't

Llamamos a la función

In [116]:
# Llamo al navegador

driver=webdriver.Chrome()
driver.get("https://www.imdb.com/?ref_=nv_home")
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

# Hago grande el navegador para que aparezcan todas las opciones posibles de la web

driver.maximize_window()

sleep(5) 

# Lista de "id" por la que quiero iterar dentro de la funcion


#lista_muestraaccion=['tt0411951','tt1652415', 'tt0423474','tt20877586','tt0429493','tt0439801','tt0446029'] #'tt0464162','tt0473075','tt0477080'] 

# Creo un archivo csv para agregar la informacion

#csv_prueba = 'csv_prueba.csv'

csv_fase2='resultados_definitivos_fase2.csv'

# Llamo a la funcion 

resultados_muestra=llamar_info_peli(lista_id_sublista,driver,csv_fase2)

driver.close()

No se ha encontrado esta película en IMDB
No se ha encontrado esta película en IMDB
No se ha encontrado esta película en IMDB
No se ha encontrado esta película en IMDB
No se ha encontrado esta película en IMDB
No se ha encontrado esta película en IMDB
No se ha encontrado esta película en IMDB
Error al procesar la búsqueda para el ID tt15404386: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__next"]/main/div[2]/div/div[2]/div/div[1]/a"}
  (Session info: chrome=124.0.6367.208); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF67CE81522+60802]
	(No symbol) [0x00007FF67CDFAC22]
	(No symbol) [0x00007FF67CCB7CE4]
	(No symbol) [0x00007FF67CD06D4D]
	(No symbol) [0x00007FF67CD06E1C]
	(No symbol) [0x00007FF67CD4CE37]
	(No symbol) [0x00007FF67CD2ABBF]
	(No symbol) [0x00007FF67CD4A224]
	(No symbol) [0x00007FF67CD2A923]

In [178]:
resultados_muestra

[('The Red, Black and Blonde: God $ave',
  'Dan Jimenez',
  'Dan Jimenez',
  'A story about four Texas women from Austin who try to rob a convenience store located in West Texas. After a night of bitching and debauchery, will they be ready for their mission in the morning?',
  82,
  '7.2'),
 ('The Last Demon Slayer',
  'Shaky González',
  'Lars Christian Detlefsen Shaky González',
  '',
  'No encontrado',
  '9.3'),
 ('Martinelli a Star Wars Fan Story',
  'Joshua Williams',
  'London Albert Joshua Williams',
  'No encontrado',
  'No encontrado',
  'No encontrado'),
 ('2 Duros de Roer',
  'Victor Santos',
  'Hugo Diogo André Mateus Fernando Rocha',
  'An unlikely team of cops and a student, live a crazy day, meet the most ridiculous characters, and unmask a drug trafficking operation with disastrous consequences not just for the city, but for the whole world.',
  80,
  '5.3'),
 ('Robin Hood',
  'Martin Schreier',
  'Martin Schreier Florian Schumacher',
  '',
  92,
  '6.1'),
 ('Xzee vs OT

In [179]:
len(resultados_muestra)

1000

AÑADO COLUMNA DE ID EN EL DICCIONARIO --->> Finalemente lo hice directamente desde el código de la función 

In [59]:
df_informacion_pelis=pd.read_csv(r'C:\Users\silvi\Desktop\Adalab\project-da-promo-H-module-2-team-Interstellar\SELENIUM\resultados_pelis_fase2.csv')

In [60]:
df_informacion_pelis

,Nombre,Director,Guionista,Argumento,Duración,Puntuación IMDb
0,"The Red, Black and Blonde: God $ave",Dan Jimenez,Dan Jimenez,A story about four Texas women from Austin who...,82,7.2
1,The Last Demon Slayer,Shaky González,Lars Christian Detlefsen Shaky González,NaN,No encontrado,9.3
2,Martinelli a Star Wars Fan Story,Joshua Williams,London Albert Joshua Williams,No encontrado,No encontrado,No encontrado
3,2 Duros de Roer,Victor Santos,Hugo Diogo André Mateus Fernando Rocha,"An unlikely team of cops and a student, live a...",80,5.3
4,Robin Hood,Martin Schreier,Martin Schreier Florian Schumacher,NaN,92,6.1
...,...,...,...,...,...,...
994,Her Dreams,No encontrado,No encontrado,No encontrado,No encontrado,No encontrado
995,Tab,Hlumela Matika,No encontrado,A coming of age story. The eldest daughter Kha...,14,5.6
996,Who Is Slim?,Dan Eccles,Jay Kwon Nathan Truong,"""Who is Slim?"" is an exciting short film with ...",No encontrado,No encontrado
997,Get Real,Darby Christensen,Darby Christensen Courtney Honors Brett Taylor,"Set in the 1980s, David is an aloof nerd who m...",45,8.4


In [61]:
# Creo diccionario de los resultados de la llamada a la función:

diccionario_info_pelis=df_informacion_pelis.to_dict() 

diccionario_info_pelis

{'Nombre': {0: 'The Red, Black and Blonde: God $ave',
  1: 'The Last Demon Slayer',
  2: 'Martinelli a Star Wars Fan Story',
  3: '2 Duros de Roer',
  4: 'Robin Hood',
  5: 'Xzee vs OTTWitt',
  6: 'Chhadke 2.0',
  7: 'Payphone',
  8: 'Cult Wars: Episode III - Sith Hits the Fan',
  9: 'La pièce rapportée',
  10: 'Babbar',
  11: 'La vicina',
  12: 'Lurk',
  13: 'Projekt Adler - Ein letzter Job',
  14: 'Operación AWOL-72',
  15: 'The Borrower',
  16: 'I Was Being Sarcastic',
  17: 'Crime Story',
  18: 'Cómo entrenar a tu dragón 2',
  19: 'Fatal instinct (Por instinto)',
  20: 'Gary Screams for You',
  21: 'Prvok, Sampon, Tecka a Karel',
  22: 'Submerged',
  23: 'Work',
  24: 'Me and the Boys',
  25: 'Something Borrowed',
  26: 'Ammonium Nitrate',
  27: 'Mercy for Angels',
  28: 'Alba',
  29: 'Naakidhe First Time',
  30: 'Forever, My Lover',
  31: 'WebSiteStory',
  32: 'Journey',
  33: 'Byt',
  34: 'The Bad Superhero',
  35: 'Up to Here',
  36: 'Tormenta infinita',
  37: 'Vardhana',
  38: 

In [39]:
# Traigo tambien la sublista de IDs, que es la columna que quiero añadir

df_sublista_pelis=pd.read_csv(r'C:\Users\silvi\Desktop\Adalab\project-da-promo-H-module-2-team-Interstellar\SELENIUM\sublista_selenium.csv')

df_sublista_pelis

,ID_Pelicula
0,tt1800301
1,tt1847719
2,tt10270648
3,tt14991492
4,tt2363363
...,...
995,tt10106152
996,tt10363182
997,tt3857098
998,tt10095498


In [55]:
# Diccionario por defecto con index
diccionario_id_index=df_sublista_pelis.to_dict() 
diccionario_id_index

{'ID_Pelicula': {0: 'tt1800301',
  1: 'tt1847719',
  2: 'tt10270648',
  3: 'tt14991492',
  4: 'tt2363363',
  5: 'tt25402886',
  6: 'tt26686370',
  7: 'tt26749149',
  8: 'tt28600432',
  9: 'tt11006570',
  10: 'tt15248456',
  11: 'tt10039168',
  12: 'tt2637906',
  13: 'tt15863566',
  14: 'tt2048865',
  15: 'tt10048402',
  16: 'tt10021686',
  17: 'tt12833220',
  18: 'tt1646971',
  19: 'tt3036654',
  20: 'tt14998762',
  21: 'tt10704730',
  22: 'tt3236976',
  23: 'tt13061874',
  24: 'tt12385404',
  25: 'tt12605454',
  26: 'tt10493472',
  27: 'tt1232828',
  28: 'tt10365862',
  29: 'tt11924222',
  30: 'tt27042672',
  31: 'tt1650461',
  32: 'tt10069486',
  33: 'tt19383308',
  34: 'tt1905034',
  35: 'tt13073642',
  36: 'tt14060232',
  37: 'tt13099072',
  38: 'tt3325766',
  39: 'tt11928632',
  40: 'tt10716904',
  41: 'tt13667552',
  42: 'tt10218912',
  43: 'tt2381249',
  44: 'tt10044678',
  45: 'tt14247286',
  46: 'tt13067244',
  47: 'tt15021154',
  48: 'tt1590796',
  49: 'tt10681656',
  50: 'tt

In [41]:
#Diccionario bien (sin index)

diccionario_id=df_sublista_pelis.to_dict(orient='list') 
diccionario_id


{'ID_Pelicula': ['tt1800301',
  'tt1847719',
  'tt10270648',
  'tt14991492',
  'tt2363363',
  'tt25402886',
  'tt26686370',
  'tt26749149',
  'tt28600432',
  'tt11006570',
  'tt15248456',
  'tt10039168',
  'tt2637906',
  'tt15863566',
  'tt2048865',
  'tt10048402',
  'tt10021686',
  'tt12833220',
  'tt1646971',
  'tt3036654',
  'tt14998762',
  'tt10704730',
  'tt3236976',
  'tt13061874',
  'tt12385404',
  'tt12605454',
  'tt10493472',
  'tt1232828',
  'tt10365862',
  'tt11924222',
  'tt27042672',
  'tt1650461',
  'tt10069486',
  'tt19383308',
  'tt1905034',
  'tt13073642',
  'tt14060232',
  'tt13099072',
  'tt3325766',
  'tt11928632',
  'tt10716904',
  'tt13667552',
  'tt10218912',
  'tt2381249',
  'tt10044678',
  'tt14247286',
  'tt13067244',
  'tt15021154',
  'tt1590796',
  'tt10681656',
  'tt1649783',
  'tt10022482',
  'tt1801128',
  'tt3361856',
  'tt1155076',
  'tt1548001',
  'tt26007064',
  'tt1647458',
  'tt10151102',
  'tt3109188',
  'tt27708536',
  'tt10411852',
  'tt10284978'

In [62]:
# Utilizo funcion update() de diccionarios para actualizar diccionario (ambos tienen que ser diccionarios)

diccionario_info_pelis.update(diccionario_id_index)

In [63]:
# Nuevo diccionario con la nueva columna

diccionario_info_pelis

{'Nombre': {0: 'The Red, Black and Blonde: God $ave',
  1: 'The Last Demon Slayer',
  2: 'Martinelli a Star Wars Fan Story',
  3: '2 Duros de Roer',
  4: 'Robin Hood',
  5: 'Xzee vs OTTWitt',
  6: 'Chhadke 2.0',
  7: 'Payphone',
  8: 'Cult Wars: Episode III - Sith Hits the Fan',
  9: 'La pièce rapportée',
  10: 'Babbar',
  11: 'La vicina',
  12: 'Lurk',
  13: 'Projekt Adler - Ein letzter Job',
  14: 'Operación AWOL-72',
  15: 'The Borrower',
  16: 'I Was Being Sarcastic',
  17: 'Crime Story',
  18: 'Cómo entrenar a tu dragón 2',
  19: 'Fatal instinct (Por instinto)',
  20: 'Gary Screams for You',
  21: 'Prvok, Sampon, Tecka a Karel',
  22: 'Submerged',
  23: 'Work',
  24: 'Me and the Boys',
  25: 'Something Borrowed',
  26: 'Ammonium Nitrate',
  27: 'Mercy for Angels',
  28: 'Alba',
  29: 'Naakidhe First Time',
  30: 'Forever, My Lover',
  31: 'WebSiteStory',
  32: 'Journey',
  33: 'Byt',
  34: 'The Bad Superhero',
  35: 'Up to Here',
  36: 'Tormenta infinita',
  37: 'Vardhana',
  38: 

In [64]:
df_resultados_completos=pd.DataFrame(diccionario_info_pelis)

In [65]:
df_resultados_completos

,Nombre,Director,Guionista,Argumento,Duración,Puntuación IMDb,ID_Pelicula
0,"The Red, Black and Blonde: God $ave",Dan Jimenez,Dan Jimenez,A story about four Texas women from Austin who...,82,7.2,tt1800301
1,The Last Demon Slayer,Shaky González,Lars Christian Detlefsen Shaky González,NaN,No encontrado,9.3,tt1847719
2,Martinelli a Star Wars Fan Story,Joshua Williams,London Albert Joshua Williams,No encontrado,No encontrado,No encontrado,tt10270648
3,2 Duros de Roer,Victor Santos,Hugo Diogo André Mateus Fernando Rocha,"An unlikely team of cops and a student, live a...",80,5.3,tt14991492
4,Robin Hood,Martin Schreier,Martin Schreier Florian Schumacher,NaN,92,6.1,tt2363363
...,...,...,...,...,...,...,...
995,Tab,Hlumela Matika,No encontrado,A coming of age story. The eldest daughter Kha...,14,5.6,tt10106152
996,Who Is Slim?,Dan Eccles,Jay Kwon Nathan Truong,"""Who is Slim?"" is an exciting short film with ...",No encontrado,No encontrado,tt10363182
997,Get Real,Darby Christensen,Darby Christensen Courtney Honors Brett Taylor,"Set in the 1980s, David is an aloof nerd who m...",45,8.4,tt3857098
998,For Christ's Sake,Jackson Douglas,Jeff Lewis,When a priest learns his brother is dying of c...,90,5.6,tt10095498


 Por último, habría que convertir la informacion completa de peliculas a CSV 

PRUEBAS PARA CONVERTIR DURACION EN MINUTOS


In [202]:
diccionario_info_pelis=df_informacion_pelis.to_dict() 

diccionario_info_pelis

{'Nombre': {0: 'The Red, Black and Blonde: God $ave',
  1: 'The Last Demon Slayer',
  2: 'Martinelli a Star Wars Fan Story',
  3: '2 Duros de Roer',
  4: 'Robin Hood',
  5: 'Xzee vs OTTWitt',
  6: 'Chhadke 2.0',
  7: 'Payphone',
  8: 'Cult Wars: Episode III - Sith Hits the Fan',
  9: 'La pièce rapportée',
  10: 'Babbar',
  11: 'La vicina',
  12: 'Lurk',
  13: 'Projekt Adler - Ein letzter Job',
  14: 'Operación AWOL-72',
  15: 'The Borrower',
  16: 'I Was Being Sarcastic',
  17: 'Crime Story',
  18: 'Cómo entrenar a tu dragón 2',
  19: 'Fatal instinct (Por instinto)',
  20: 'Gary Screams for You',
  21: 'Prvok, Sampon, Tecka a Karel',
  22: 'Submerged',
  23: 'Work',
  24: 'Me and the Boys',
  25: 'Something Borrowed',
  26: 'Ammonium Nitrate',
  27: 'Mercy for Angels',
  28: 'Alba',
  29: 'Naakidhe First Time',
  30: 'Forever, My Lover',
  31: 'WebSiteStory',
  32: 'Journey',
  33: 'Byt',
  34: 'The Bad Superhero',
  35: 'Up to Here',
  36: 'Tormenta infinita',
  37: 'Vardhana',
  38: 

In [217]:
diccionario_info_pelis.keys()

dict_keys(['Nombre', 'Director', 'Guionista', 'Argumento', 'Duración', 'Puntuación IMDb'])

In [204]:
horas_pelis=diccionario_info_pelis["Duración"]

In [132]:
horas_pelis_values=horas_pelis.values()
lista_horas_pelis_values=list(horas_pelis_values)
lista_horas_pelis_values

['1h 22min',
 'No encontrado',
 'No encontrado',
 '1h 20min',
 '1h 32min',
 'No encontrado',
 'No encontrado',
 '19min',
 'No encontrado',
 '1h 26min',
 '2h 5min',
 '5min',
 '1h 25min',
 '1h 40min',
 '1h 22min',
 '4min',
 'No encontrado',
 '2h 20min',
 '1h 42min',
 '1h 30min',
 '15min',
 '1h 58min',
 '1h 39min',
 'No encontrado',
 'No encontrado',
 'No encontrado',
 'No encontrado',
 '1h 31min',
 'No encontrado',
 'No encontrado',
 'No encontrado',
 '1h 22min',
 '11min',
 '1h 54min',
 '5min',
 '7min',
 '1h 37min',
 '2h 10min',
 '1h 13min',
 '1h 32min',
 'No encontrado',
 '49min',
 '1h 2min',
 '2h 11min',
 '15min',
 '1h 45min',
 'No encontrado',
 '1h 42min',
 '1h 29min',
 'No encontrado',
 '2h 20min',
 '11min',
 '13min',
 '1h 30min',
 '2h 20min',
 '1h 38min',
 'No encontrado',
 '1h 42min',
 'No encontrado',
 '1h 21min',
 'No encontrado',
 'No encontrado',
 'No encontrado',
 '1h 45min',
 'No encontrado',
 'No encontrado',
 '28min',
 'No encontrado',
 '1h 23min',
 'No encontrado',
 '2h',


In [139]:
#diccionario_info_pelis["Duración"][0] 

elementos=lista_horas_pelis_values[0]
elementos

'1h 22min'

In [141]:
#REGEX

#numeros = re.findall("\d+", string)

numeros_duracion=re.findall("\d+", elementos)
numeros_duracion


['1', '22']

In [151]:
horas= numeros_duracion[0]

minutos= (int(horas)*60)
minutos

duracion_minutos=int(minutos)+int(numeros_duracion[1])
duracion_minutos

82